# 빅데이터분석 기말 프로젝트

* 분석 데이터 : POS_CASH_balance / bureau
* EDA / feature engineering / LGBMClassifier
* 수업에서 다루지 않았던 pos_cash_balance데이터와 함께 bureau를 분석 

**라이브러리 세팅 **

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 라이브러리 및 데이터 세트 로딩
import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

# 데이터 준비 : applicaiont 데이터 통합

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
# app_train 과 app_test를 함께 가공을 해야하기 때문에 concat 이용하여 통합

apps = pd.concat([app_train, app_test])

# 활용 데이터 - 1. POS_CASH_balance

In [ ]:
pos = pd.read_csv('../input/home-credit-default-risk/POS_CASH_balance.csv')

In [ ]:
#pos - app 데이터 조인
#pos_app_outer = pos.merge(apps['SK_ID_CURR'], on='SK_ID_CURR', how='outer', indicator=True)

In [ ]:
# Null 값을 확인하고 전체 데이터에서 비율을 보여주는 함수

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
#pos_app_outer.info()
# object data는 1개이며, 나머지 데이터는 모두 수치형 데이터

In [ ]:
missing_data(pos).head(20)
#missing 값이 전체적으로 크지 않다

In [ ]:
pos.groupby('SK_ID_CURR').count()
# pos - 숫자 분포도 고른 것으로 보임

In [ ]:
pos.groupby('SK_ID_CURR')['SK_ID_CURR'].count().mean()
#ID를 통한 데이터 평균 개수 확인

In [ ]:
#pos_app_outer.describe()
#pos-app 통합데이터 수치적 특성 확인

# EDA : POS_CASH
* SK_ID_CURR로 applicaion data와 연결
* 현 월 대출 상태 : NAME_CONTRACT_STATUS 만 object data / 수치형 위주로 탐색
* POS_CASH를 이용한 가설 :

In [ ]:
app_pos_target = pos.merge(app_train[['SK_ID_CURR', 'TARGET']], on='SK_ID_CURR', how='left')
app_pos_target.shape

In [ ]:
app_pos_target.head()
#status만 categorical이다.

In [ ]:
missing_data(app_pos_target).head(20)

# POS-EDA : object 형 데이터

In [ ]:
# 시각화 함수
def plot_re(df,t1='',t2=''):
    f,ax=plt.subplots(1,2,figsize=(12,8))
    df[[t1,t2]].groupby([t1]).count().plot.bar(ax=ax[0],color='Blue')
    ax[0].set_title('count of customer on '+t1)
    sns.countplot(t1,hue=t2,data=df,ax=ax[1],palette="spring")
    ax[1].set_title(t1+': Target 0 vs Target 1')
    # Rotate x-labels
    plt.xticks(rotation=-90)
    a=plt.show()
    return a

In [ ]:
# object형 데이터 추출

obj_columns = app_pos_target.dtypes[app_pos_target.dtypes == 'object'].index.tolist()
obj_columns

In [ ]:
#현월 대출 상태에 따른 TARGET값
plot_re(app_pos_target, 'NAME_CONTRACT_STATUS','TARGET')

#Active일 경우에 Target이 0일 것으로 예상했지만, 대출 여부에 대한 기록이기 때문에 TARGET 0이 대체로 많이 분포하는 것으로 보임
#단순 대출 상태로는 TARGET과 크게 연관짓기는 어려워 보임

In [ ]:
#sns활용해서 그려보기
#chart=sns.catplot(x='NAME_CONTRACT_STATUS', col='TARGET', data=app_pos_target, kind='count')

#chart.set_xticklabels(chart.get_xticklabels(), rotation=45)

In [ ]:
# 그래프에서 경미한 차이이기 때문에 수치상으로도 살펴본다
pd.crosstab(app_pos_target.TARGET, app_pos_target.NAME_CONTRACT_STATUS, dropna=False, normalize='all')

#Active/ Completed가 비례하여 TARGET 0 / 1에 차이를 보이지는 않지만 TARGET0이 월등히 높다

# POS-EDA : 수치형 데이터

In [ ]:
#TARGET 0 : blue / TARGET 1: red
def show_hist_by_target(df, columns):
    cond_1 = (df['TARGET'] == 1)
    cond_0 = (df['TARGET'] == 0)
    
    for column in columns:
        fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 4), squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=axs[0][0] )
        sns.distplot(df[cond_0][column], ax=axs[0][1], label='0', color='blue')
        sns.distplot(df[cond_1][column], ax=axs[0][1], label='1', color='red')

In [ ]:
# 숫자형 dtype 갖고 있는 컬럼만 추출

num_columns = app_pos_target.dtypes[app_pos_target.dtypes != 'object'].index.tolist()
num_columns

In [ ]:
# 필요한 columns만 추출

num_columns = [column for column in num_columns if column not in ['SK_ID_PREV', 'SK_ID_CURR', 'TARGET','_merge']]
num_columns

In [ ]:
plt.figure(figsize=(10,12))

# iterate through the sources
for i, source in enumerate(['MONTHS_BALANCE','CNT_INSTALMENT','CNT_INSTALMENT_FUTURE','SK_DPD','SK_DPD_DEF']):
    plt.subplot(5,1,i+1)
    
    sns.kdeplot(app_pos_target.loc[app_pos_target['TARGET']==0,source],label='target==0')
    sns.kdeplot(app_pos_target.loc[app_pos_target['TARGET']==1,source],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');
plt.tight_layout(h_pad=2.5)


# 발견: MONTHS BALANCE와 CNT_INSTALLMENT_FUTURE은 x축 증감에 대한 대칭된 흐름을 가지고 있다 -> 상관관계로 확인해보자
# 완만한 흐름보다는, 데이터 특성상 값이 몰리는 경우가 많은 것 같다.

#CNT_INSTALMENT에 대해 과거 납입 횟수와 납입 예정(FUTURE)의 요소는 서로 관련있는 데이터 임에도 꽤 다른 분포를 확인-> 이미 이행한 요소보다 이행할 연체가 영향을 크게 미치는듯
#CNT_FUTURE의 경우 납입예정 횟수가 작을 수록 TARGET값의 큰 변화가 관찰됨
#CNT_INSTALMENT는 20단위로 그래프가 나뉘어 지는 섹션내에서 꼭지점의 개수가 줄고, 변화폭도 줄어든다. -> 특히 20이전-40구간에서 약한 패턴이 관찰

#가설 : 과거 연체 여부보다 미래에 관한 연체 부담의 결과가 중요한 것 같다

#다른 csv파일에서 연체에 관해 유사한 흐름을 나타내는 바가 있는지 살펴봐도 좋을 것 같음. -> bureau

In [ ]:
#다른 방법 그래프

'''f, ax = plt.subplots(5,1,figsize=(13, 10))

# SK_DPD: 연체 일자 분포 확인
sns.distplot(app_pos_target.SK_DPD.dropna(), kde=True, color="g", 
             ax=ax[0]).set_title('SK_DPD Distribution')


# MONTHS_BALANCE: 신청일 기준 잔액 월(-1은 가장 최근 잔액 월 의미)

sns.distplot(app_pos_target.MONTHS_BALANCE.dropna(), kde=True, color="g", 
             ax=ax[1]).set_title('MONTHS_BALANCE Distribution')



# CNT_INSTALMENT: 납부 횟수
sns.distplot(app_pos_target.CNT_INSTALMENT.dropna(), kde=True, color="g", 
             ax=ax[2]).set_title('CNT_INSTALMENT Distribution')


# CNT_INSTALMENT: 납부 해야할 횟수
sns.distplot(app_pos_target.CNT_INSTALMENT_FUTURE.dropna(), kde=True, color="g", 
             ax=ax[3]).set_title('CNT_INSTALMENT_FUTURE Distribution')


# SK_DPD_DEF: 월에 허용가능한 연체 일자
sns.distplot(app_pos_target.SK_DPD_DEF.dropna(), kde=True, color="g", 
             ax=ax[4]).set_title('SK_DPD_DEF Distribution')
'''



In [ ]:
#위의 그래프로 유의함을 확인하였던 변수 상관관계

ext_data=app_pos_target[['MONTHS_BALANCE','CNT_INSTALMENT','CNT_INSTALMENT_FUTURE','SK_DPD','SK_DPD_DEF','TARGET']]
ext_data_corrs=ext_data.corr()
ext_data_corrs



In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

# 그래프에서 대칭 흐름을 보였던 Month balance-INSTALMENT 상대적으로 유의한 관계성을 보임
#INSTALMENT-FUTURE개별적인 분포를 확인해도 좋을듯
#SK_DPD / DEF 높은 편
#-> feature engineering 에 참고하면 좋음

In [ ]:
# 'CNT_INSTALMENT_FUTURE' : 앞으로 남아있는 납부 횟수

app_pos_target.groupby(['NAME_CONTRACT_STATUS','TARGET']).agg({'CNT_INSTALMENT_FUTURE': ['mean', 'median', 'count', 'max']})
# TARGET/ 현월 대출 상태에 따른 납부 예정 횟수

In [ ]:
# 신청자의 대출 진행상황 : 현재 상태에 비추어 완료한 / 예정인 대출 현황을 구해보자 -> 현재 연체 부담율에 연관되지 않을까?
app_pos_target['CNT_INSTALMENT_RATIO']=(app_pos_target['CNT_INSTALMENT']-app_pos_target['CNT_INSTALMENT_FUTURE'])/app_pos_target['CNT_INSTALMENT']*100
app_pos_target

In [ ]:
#그래프 스케일 조절을 위해 양수만 포함

condition_1 = app_pos_target['CNT_INSTALMENT_RATIO'] > 0

In [ ]:

# CNT INSTALMENT RATIO 분포 확인
#sns.distplot(df.cond1.dropna(), kde=True, color="r").set_title('CNT INSTALMENT RATIO')


sns.distplot(app_pos_target[condition_1]['CNT_INSTALMENT_RATIO'], kde=True, color="g").set_title('RATE DOWN PAYMENT Distribution')
#증감이 꾸준히 관찰되지만 눈에 띄는 경향성은 보이지 않는다

In [ ]:
# 놓친 특징이 있는지 수치로 체크
print(app_pos_target.groupby('TARGET').agg({'MONTHS_BALANCE': ['mean', 'median', 'count','sum','max']}))
print("*"*60)
print(app_pos_target.groupby('TARGET').agg({'CNT_INSTALMENT': ['mean', 'median', 'count','sum','max']}))
print("*"*60)
print(app_pos_target.groupby('TARGET').agg({'CNT_INSTALMENT_FUTURE': ['mean', 'median', 'count','sum','max']}))
print("*"*60)
print(app_pos_target.groupby('TARGET').agg({'SK_DPD': ['mean', 'median', 'count','sum','max']}))
print("*"*60)
print(app_pos_target.groupby('TARGET').agg({'SK_DPD_DEF': ['mean', 'median', 'count','sum','max']}))

#대부분의 faeture가 TARGET값에 따라 큰차이는 없으나 SK_DPD / SK_DPD_DEF가 차이를 보인다 -> feature check

In [ ]:
# 놓친 특징이 있는지 그래프로 체크
show_hist_by_target(app_pos_target, num_columns)

# Feature Engineering - POS_CASH

In [ ]:
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns

def do_sum(df, group_cols, counted, agg_name):
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].sum().reset_index().rename(
        columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

In [ ]:
#categorical 처리
pos, categorical_cols = one_hot_encoder(pos, nan_as_category= False)

In [ ]:
#SK_PDP로 LATE PAYMENT 여부 feature 생성
pos['LATE_PAYMENT'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)



In [ ]:
pos.head()

In [ ]:
# aggregations 선언
aggregations = {
    'MONTHS_BALANCE': ['max', 'mean', 'size', 'min'],
    'SK_DPD': ['max', 'mean', 'sum', 'var', 'min'],
    'SK_DPD_DEF': ['max', 'mean', 'sum'],
    'SK_ID_PREV': ['nunique'],
    'LATE_PAYMENT': ['mean'],
    'SK_ID_CURR': ['count'],
    'CNT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
    'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean', 'sum']
}

In [ ]:
#aggregation진행
pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])

# POS_COUNT기록 
pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()

#그래프상 유의하였던 MONTHS_BALANCE의 값으로 feature를 생성
sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
gp = sort_pos.groupby('SK_ID_PREV')
df_pos = pd.DataFrame()
df_pos['SK_ID_CURR'] = gp['SK_ID_CURR'].first()
df_pos['MONTHS_BALANCE_MAX'] = gp['MONTHS_BALANCE'].max()



In [ ]:
# complete된 loan에 대하여 처음 instalment와 first/last를 활용하여 feature생성
df_pos['POS_LOAN_COMPLETED_MEAN'] = gp['NAME_CONTRACT_STATUS_Completed'].mean()
df_pos['POS_COMPLETED_BEFORE_MEAN'] = gp['CNT_INSTALMENT'].first() - gp['CNT_INSTALMENT'].last()


# 그래프에서 유의했던 INSTALMENT/FUTURE 관련 feature
df_pos['POS_REMAINING_INSTALMENTS'] = gp['CNT_INSTALMENT_FUTURE'].last()
df_pos['POS_REMAINING_INSTALMENTS_RATIO'] = gp['CNT_INSTALMENT_FUTURE'].last()/gp['CNT_INSTALMENT'].last()




In [ ]:
# SK_ID merge
df_gp = df_pos.groupby('SK_ID_CURR').sum().reset_index()
df_gp.drop(['MONTHS_BALANCE_MAX'], axis=1, inplace= True)
pos_agg = pd.merge(pos_agg, df_gp, on= 'SK_ID_CURR', how= 'left')
del gp, df_gp, sort_pos; gc.collect()


# 최근 applications에 대한 late payment비율
pos = do_sum(pos, ['SK_ID_PREV'], 'LATE_PAYMENT', 'LATE_PAYMENT_SUM')
#각 application에 대한 lastmonth
last_month_df = pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()



In [ ]:
# 최근 데이터를 활용하여 feature를 생성한다 (last 3)
sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
gp = sort_pos.iloc[last_month_df].groupby('SK_ID_CURR').tail(3)
gp_mean = gp.groupby('SK_ID_CURR').mean().reset_index()
pos_agg = pd.merge(pos_agg, gp_mean[['SK_ID_CURR','LATE_PAYMENT_SUM']], on='SK_ID_CURR', how='left')


#pos_agg 최종 

# 연체와 관련하여 bureau를 추가

* POS_CASH로 인한 가설: POS_CASH에 의하면 과거의 연체여부보다 앞으로 이행될 연체여부가 TARGET에 영향을 많이 줄 것이다.
* credit/installment등의 데이터가 있지만 메모리 문제로 bureau를 활용

In [ ]:
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')

In [ ]:
# TARGET 값을 가져오기 위해 bureau를 apps와 조인
app_bureau = bureau.merge(app_train[['SK_ID_CURR', 'TARGET']], left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')
app_bureau.shape

In [ ]:
#target비율 확인을 위한 그래프
f,ax=plt.subplots(1,2,figsize=(12,6))
app_train.TARGET.value_counts().plot.pie(explode=[0,0.1],autopct='%1.2f%%',ax=ax[0],shadow=True)
ax[0].set_title('Distribution of Target')
ax[0].set_ylabel('')
sns.countplot('TARGET',data=app_train,ax=ax[1])
ax[1].set_title('Target count')
plt.show()

In [ ]:
#CREDIT 종류별 확인
plot_re(app_bureau,'CREDIT_TYPE','TARGET')

# POS_CASH와 연관하여 현금 LOAN에 대해 확인하기 힘들다.

In [ ]:
#수치로 확인
app_bureau.groupby('CREDIT_TYPE').count()

#CASH loan자체 분포 숫자가 적은편, 따라서 POS_CASH와의 직접적인 비교는 불가하고, 다른 loan type으로 간접적으로 살펴봐야함


In [ ]:
# 함수 호출 
# 대출 상태에 따른 Target
plot_re(app_bureau,'CREDIT_ACTIVE','TARGET')
#대부분의 카테고리의 경우 TARGET0이 월등하게 높은편. 비율상 Closed가 더욱 높다.

In [ ]:
#BREAU에서 관련있는 데이터 
plt.figure(figsize=(10,12))

for i, source in enumerate(['DAYS_CREDIT_ENDDATE','AMT_CREDIT_SUM_DEBT','DAYS_ENDDATE_FACT']):
    plt.subplot(3,1,i+1)
    sns.kdeplot(app_bureau.loc[app_bureau['TARGET']==0, source], label='target==0', color='r')
    sns.kdeplot(app_bureau.loc[app_bureau['TARGET']==1, source], label='target==1', color='b')
    
    plt.title('distribution of %s by Traget Value' %source)
    plt.xlabel('%s' %source)
    plt.ylabel('Density')

plt.tight_layout(h_pad=2.5)

#POS에서 제안된 가설로 ENDDATE가 미래에 관한 예측데이터와 관련있다고 할 수 있다.
#DAYS_CREDIT_ENDDATE: 남아있는 일수 : 0을 기준으로 가장큰 변화폭을 가지고 있다 : 미세하게 0을 기준으로 0보다 작을경우 TARGET 0 > TARGET 1 이다.
#DAYS_ENDDATE_FACT: 완료까지 남아있는 일수 : 0과 가까울수록 TARGET1이 눈에 띄게 증가하지만 그 이전에는 TARGET0이 더 많다.

In [ ]:
#ax = sns.scatterplot(x='CNT_CREDIT_PROLONG', y='AMT_CREDIT_MAX_OVERDUE', hue='TARGET', data=app_bureau)

In [ ]:
#TARGET0이 미세하게 더 분포되어 있다.
sns.stripplot(y = app_bureau['AMT_CREDIT_SUM_DEBT'], x = app_bureau['TARGET'])

In [ ]:
f, ax = plt.subplots(2,3,figsize=(13, 10))


# DAYS_CREDIT: 현재 대출 신청 일 기준 과거 대출 신청 지난 기간 분포 확인
sns.distplot(app_bureau.DAYS_CREDIT.dropna(), kde=True, color="g", 
             ax=ax[0,0]).set_title('DAYS CREDIT Distribution')

# CREDIT_DAY_OVERDUE: 대출 신청 시 CB 크레딧 연체 일수
sns.distplot(app_bureau.CREDIT_DAY_OVERDUE.dropna(), kde=True, color="b",
             ax=ax[0,1]).set_title(' CREDIT DAY OVERDUE Distribution')


# DAYS_CREDIT_UPDATE: 대출 신청전 마지막 정보 받은 기간
sns.distplot(bureau.DAYS_CREDIT_UPDATE.dropna(), kde=True, 
             color="r", ax=ax[0,2]).set_title('DAYS CREDIT UPDATE Distribution')


# AMT_CREDIT_SUM_LIMIT: 신용 카드 현재 신용한도
sns.distplot(bureau.AMT_CREDIT_SUM_LIMIT.dropna(), kde=True, color="g", 
             ax=ax[1,0]).set_title(' Distribution')


# AMT_CREDIT_SUM_DEBT: 현재 채무 금액 총액
sns.distplot(bureau.AMT_CREDIT_SUM_DEBT.dropna(), kde=True, color="b",
             ax=ax[1,1]).set_title(' Distribution')

 
# AMT_CREDIT_SUM_OVERDUE: 최대 연체금액
sns.distplot(bureau.AMT_CREDIT_SUM_OVERDUE.dropna(), kde=True, 
             color="r", ax=ax[1,2]).set_title('DAYS CREDIT UPDATE Distribution')

# DAYS CREDIT DISTRIBUTION의 경우 대체적으로 x값이 줄어들면서 Density 또한 줄어드는 추세이다.

# Feature Engineering : beurue

In [ ]:

# ID당 과거 이력 종류를 feature행

grp = app_bureau[['SK_ID_CURR', 'CREDIT_TYPE']].groupby(by = ['SK_ID_CURR'])['CREDIT_TYPE'].nunique().reset_index().rename(index=str, columns={'CREDIT_TYPE': 'BUREAU_LOAN_TYPES'})
app_bureau = app_bureau.merge(grp, on = ['SK_ID_CURR'], how = 'left')


In [ ]:
# ACTIVE / 아님에 관한 feature -> 그래프 상에서 관련성을 보였음
app_bureau['CREDIT_ACTIVE_BINARY'] = app_bureau['CREDIT_ACTIVE']

def f(x):
    if x == 'Closed':
        y = 0
    else:
        y = 1    
    return y

app_bureau['CREDIT_ACTIVE_BINARY'] = app_bureau.apply(lambda x: f(x.CREDIT_ACTIVE), axis = 1)

# Active에 관한 평균에 대한 feature를 생성
grp = app_bureau.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})
app_bureau = app_bureau.merge(grp, on = ['SK_ID_CURR'], how = 'left')
del app_bureau['CREDIT_ACTIVE_BINARY']
import gc
gc.collect()
app_bureau

In [ ]:


# ASCENDING 순으로 DAYS_CREDIT -> POS_CASH가설에서 넘어온 feature 생성
grp = app_bureau[['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT']].groupby(by = ['SK_ID_CURR'])
grp1 = grp.apply(lambda x: x.sort_values(['DAYS_CREDIT'], ascending = False)).reset_index(drop = True)#rename(index = str, columns = {'DAYS_CREDIT': 'DAYS_CREDIT_DIFF'})
print("Grouping and Sorting done")

#날짜 차이로 feature 생성
grp1['DAYS_CREDIT1'] = grp1['DAYS_CREDIT']*-1
grp1['DAYS_DIFF'] = grp1.groupby(by = ['SK_ID_CURR'])['DAYS_CREDIT1'].diff()
grp1['DAYS_DIFF'] = grp1['DAYS_DIFF'].fillna(0).astype('uint32')
del grp1['DAYS_CREDIT1'], grp1['DAYS_CREDIT'], grp1['SK_ID_CURR']
gc.collect()
print("Difference days calculated")

app_bureau = app_bureau.merge(grp1, on = ['SK_ID_BUREAU'], how = 'left')
app_bureau

In [ ]:
# 연체 지속 / end date관련 feature생성
app_bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
app_bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']


# CREDIT_DAY_OVERDUE 관련 feature
app_bureau['BUREAU_IS_DPD'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)
app_bureau['BUREAU_IS_DPD_OVER120'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 120 else 0)


# 모델링 : 수업과정 참조

In [ ]:
def get_apps_processed(apps):
    
    # EXT_SOURCE_X FEATURE 가공
    apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())
    
    # AMT_CREDIT 비율로 Feature 가공
    apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
    apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
    
    # AMT_INCOME_TOTAL 비율로 대출 금액 관련 Feature 가공
    apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
    apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']
    
    # DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공
    apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
    apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
    apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
    apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
    apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']
    
    
    
    
     # Credit ratios
    apps['CREDIT_TO_ANNUITY_RATIO'] = apps['AMT_CREDIT'] / apps['AMT_ANNUITY']

    
    return apps

In [ ]:
#apps_all =  get_apps_processed(apps)

#CPU문제로 자르고고 진행합니다
#app_bureau = app_bureau[0:1000000] -> CPU문제로 모델링에서는 활용하지 못할 것 같습니다..
pos_agg = pos_agg[0:1300000]
gc.collect()

In [ ]:
'''print(apps.shape, app_bureau.shape)
apps_all = apps.merge(app_bureau, on='SK_ID_CURR', how='left')
print(apps_all.shape)'''
#-> CPU문제로 모델링에서는 활용하지 못할 것 같습니다..

In [ ]:
apps_all = apps.merge(pos_agg, on='SK_ID_CURR', how='left')
#apps_all = apps_all.merge(pos_agg, on='SK_ID_CURR', how='left')

In [ ]:
apps_all

In [ ]:
#breau사용시 필요
'''apps_all = apps_all.rename(columns = {"TARGET_x" : "TARGET"})
apps_all'''

In [ ]:
object_columns = apps_all.dtypes[apps_all.dtypes == 'object'].index.tolist()
for column in object_columns:
    apps_all[column] = pd.factorize(apps_all[column])[0]

In [ ]:
apps_all

In [ ]:
apps_all_train = apps_all[~apps_all['TARGET'].isnull()]
apps_all_test = apps_all[apps_all['TARGET'].isnull()]

apps_all_test = apps_all_test.drop('TARGET', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = apps_all_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 50)

In [ ]:
from lightgbm import plot_importance

plot_importance(clf, figsize=(16, 32))

In [ ]:
preds = clf.predict_proba(apps_all_test.drop('SK_ID_CURR', axis=1))[:, 1 ]
apps_all_test['TARGET'] = preds
apps_all_test[['SK_ID_CURR', 'TARGET']].to_csv('app_pos_breau.csv', index=False)

* 참조문헌

https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction
https://www.kaggle.com/codename007/home-credit-complete-eda-feature-importance
https://www.kaggle.com/ogrellier/lighgbm-with-selected-features